# Regression test suite for the Swath Projector service:

This notebook provides condensed examples of using Harmony to make requests against the [Swath Projector service]() developed and managed by the Data Services team on the Transformation Train. This service take input swath data and projects the variables within that swath to a grid as defined by the input Harmony request.

A request to the Swath Projector can specify a number of parameters, or combinations thereof:

* A target projection specified by an EPSG code (e.g., "EPSG:4326" for geographically gridded data).
* A target projection specified as a Proj4 string (e.g., "+proj=lcc +lat_1=43 +lat_2=62 +lat_0=30 +lon_0=10 +x_0=0 +y_0=0 +ellps=intl +units=m +no_defs").
* A target grid height and width (e.g., the number of pixel columns and rows).
* A target grid resolution (in degrees or projected metres).
* A target grid extent (e.g., the longitude and latitude or x and y ranges of the output grid).

Note, several configuration tips were gained from [this blog post](https://towardsdatascience.com/introduction-to-papermill-2c61f66bea30).

## Prerequisites

The dependencies for this notebook are listed in the [environment.yaml](./environment.yaml). To test or install locally, create the papermill environment used in the automated regression testing suite:

`conda env create -f ./environment.yaml && conda activate papermill`

A `.netrc` file must also be located in the `test` directory of this repository.

## Import required packages:

In [ ]:
from datetime import datetime
from os.path import exists

from harmony import Client, Collection, Environment, Request

from utilities import (compare_results_to_reference_file, print_success,
                       remove_results_files, submit_and_download)

## Set default parameters:

`papermill` requires default values for parameters used on the workflow. In this case, `harmony_host_url`.

In [ ]:
harmony_host_url = 'https://harmony.uat.earthdata.nasa.gov'

### Identify Harmony environment (for easier reference):

In [ ]:
host_environment = {'http://localhost:3000': Environment.LOCAL,
                    'https://harmony.sit.earthdata.nasa.gov': Environment.SIT,
                    'https://harmony.uat.earthdata.nasa.gov': Environment.UAT,
                    'https://harmony.earthdata.nasa.gov': Environment.PROD}


harmony_environment = host_environment.get(harmony_host_url)

if harmony_environment is not None:
    harmony_client = Client(env=harmony_environment)

# Begin regression tests:

The Swath Projector is currently only configured against collections in the SIT and UAT environment. These tests use the [Harmony L2 example data collection](https://cmr.uat.earthdata.nasa.gov/search/concepts/C1233860183-EEDTEST.html).

In [ ]:
swath_projector_non_prod_information = {'collection': Collection(id='C1233860183-EEDTEST'),
                                        'granule_id': 'G1233860549-EEDTEST'}

swath_projector_env = {Environment.LOCAL: swath_projector_non_prod_information,
                       Environment.SIT: swath_projector_non_prod_information,
                       Environment.UAT: swath_projector_non_prod_information}

if harmony_environment in swath_projector_env:
    swath_projector_info = swath_projector_env[harmony_environment]
else:
    swath_projector_info = None

### Swath Projector request with defaults:

Make a request that only specifies the collection and an appropriate granule. This should rely on the default target Coordinate Reference System (CRS) and interpolation method.

In [ ]:
if swath_projector_info is not None:
    defaults_file_name = 'swath_projector_defaults.nc4'
    defaults_request = Request(collection=swath_projector_info['collection'],
                               granule_id=[swath_projector_info['granule_id']])

    submit_and_download(harmony_client, defaults_request, defaults_file_name)
    assert exists(defaults_file_name), 'Unsuccessful Swath Projector defaults request.'

    compare_results_to_reference_file(defaults_file_name,
                                      'reference_files/swath_projector_defaults_reference.nc4')

    print_success('Swath projector with default parameters.')
else:
    print(f'The Swath Projector is not configured for environment: "{harmony_environment}" - skipping test.')

### Swath Projector request for Madagascar:

Make a request to the Swath Projector specifying a target CRS using an EPSG code, and requested that the target grid covers only the area surrounding Madagascar, using the `scaleExtents` parameter.

In [ ]:
if swath_projector_info is not None:
    epsg_file_name = 'swath_projector_epsg.nc4'
    epsg_request = Request(collection=swath_projector_info['collection'],
                           granule_id=[swath_projector_info['granule_id']],
                           crs='EPSG:4326', scale_extent=[42, -27, 52, -10],
                           temporal={'start': datetime(2020, 1, 15), 'stop': datetime(2020, 1, 16)})

    submit_and_download(harmony_client, epsg_request, epsg_file_name)
    assert exists(epsg_file_name), 'Unsuccessful Swath Projector EPSG code request.'

    compare_results_to_reference_file(epsg_file_name,
                                      'reference_files/swath_projector_epsg_reference.nc4')

    print_success('Swath Projector EPSG code request.')
else:
    print(f'The Swath Projector is not configured for environment: "{harmony_environment}" - skipping test.')

### Swath Projector, interpolation type and Proj4:

Use the `interpolation` and `outputCrs` parameters to ensure a raw Proj4 string is valid input and that the user can select a non-default interpolation type.

In [ ]:
if swath_projector_info is not None:
    proj4_string_file_name = 'swath_projector_proj4.nc4'
    proj4_lcc = '+proj=lcc +lat_1=43 +lat_2=62 +lat_0=30 +lon_0=10 +x_0=0 +y_0=0 +ellps=intl +units=m +no_defs'
    proj4_string_request = Request(collection=swath_projector_info['collection'],
                                   granule_id=[swath_projector_info['granule_id']],
                                   crs=proj4_lcc, interpolation='near',
                                   temporal={'start': datetime(2020, 1, 15), 'stop': datetime(2020, 1, 16)})

    submit_and_download(harmony_client, proj4_string_request, proj4_string_file_name)
    assert exists(proj4_string_file_name), 'Unsuccessful Swath Projector interpolation and Proj4 request.'

    compare_results_to_reference_file(proj4_string_file_name,
                                      'reference_files/swath_projector_proj4_reference.nc4')

    print_success('Swath Projector interpolation and Proj4 request')
else:
    print(f'The Swath Projector is not configured for environment: "{harmony_environment}" - skipping test.')

## Clean up results files:

In [ ]:
remove_results_files()